In [2]:
import pandas as pd
from google.colab import files
uploaded = files.upload()



Saving data_stocks.csv to data_stocks.csv


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
df = pd.read_csv('data_stocks.csv')
print (df)

             DATE      SP500  NASDAQ.AAL  NASDAQ.AAPL  NASDAQ.ADBE  \
0      1491226200  2363.6101     42.3300     143.6800     129.6300   
1      1491226260  2364.1001     42.3600     143.7000     130.3200   
2      1491226320  2362.6799     42.3100     143.6901     130.2250   
3      1491226380  2364.3101     42.3700     143.6400     130.0729   
4      1491226440  2364.8501     42.5378     143.6600     129.8800   
5      1491226500  2365.6201     42.5399     143.7800     130.0700   
6      1491226560  2365.2000     42.4700     143.8640     130.1800   
7      1491226620  2365.2900     42.4700     143.8100     130.1400   
8      1491226680  2364.3201     42.3900     143.8150     130.1000   
9      1491226740  2364.6399     42.3300     143.8000     130.2100   
10     1491226800  2364.5801     42.4000     143.8900     130.1400   
11     1491226860  2364.4800     42.2900     143.9700     130.3700   
12     1491226920  2365.2300     42.2900     143.9199     130.4599   
13     1491226980  2

In [0]:

df = df.drop(['DATE'], 1)

# dimensions of dataset
n = df.shape[0]
p = df.shape[1]

df = df.values



In [0]:
# preparing training (80%), test data (20%) - April - August 2017 S&P 500 stock price data

#time series prediction data

train_start = 0 
train_end = int(np.floor(0.8*n))
test_start = train_end
test_end = n
df_train = df[np.arange(train_start, train_end), :]
df_test = df[np.arange(test_start, test_end), :]


In [0]:
# Scaling the inputs, targets, to allow for activation functions on networks neurons which are bounded to work 

scaler = MinMaxScaler()
df_train = scaler.fit_transform(df_train)
df_test = scaler.fit_transform(df_test)

# build x and y, do not scale whole data set yet, as in real life time 
# series prediction we don't have future observations at the time of 
# forecasting

x_train = df_train[:, 1:]
y_train = df_train[: 0]
x_test = df_test[:, 1:]
y_test = df_test [:, 0]


In [0]:
n_stocks = x_train.shape[1]
X = tf.placeholder(dtype=tf.float32, shape = [None, n_stocks])
Y = tf.placeholder(dtype=tf.float32, shape = [None])

In [0]:
n_stocks = 500 
n_neurons_1 = 1024 # 1st layer, 2nd layer, always half of the previous layer 
n_neurons_2 = 512
n_neurons_3 = 256
n_neurons_4 = 128

# Session variable
net = tf.InteractiveSession()

# Initiailizers used in adjusting weights and bioases

sigma = 1 # scale
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale = sigma)
bias_initializer = tf.zeros_initializer()

# weights and biases, second dimension of previous layer = first dimesion of current layer

# layer 1 : weights and biases initialized

weights_hidden_1 = tf.Variable(weight_initializer([n_stocks, n_neurons_1])) # input to first layer
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))

# layer 2 : weights and biases initialized
weights_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))

# layer 3 : weights and biases initialized
weights_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))

# layer 4 : weights and biases initialized
weights_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
# let's initialize some output weights! 
weight_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))


In [0]:
# specified network architecture - combining data and weights and biases for matrix multiplication
# use ReLU activation function for model 

# feedforward network - data flows from left to right (input layer, hidden layers, output layers)
# hidden layers 
hidden_layer1 = tf.nn.relu(tf.add(tf.matmul(X, weights_hidden_1), bias_hidden_1)) 
dropped = tf.nn.dropout(hidden_layer1, keep_prob = 0.5)
hidden_layer2 = tf.nn.relu(tf.add(tf.matmul(hidden_layer1, weights_hidden_2), bias_hidden_2))
dropped = tf.nn.dropout(hidden_layer2, keep_prob = 0.5)
hidden_layer3 = tf.nn.relu(tf.add(tf.matmul(hidden_layer2, weights_hidden_3), bias_hidden_3))
dropped = tf.nn.dropout(hidden_layer3, keep_prob = 0.5)
hidden_layer4 = tf.nn.relu(tf.add(tf.matmul(hidden_layer3, weights_hidden_4), bias_hidden_4))
dropped = tf.nn.dropout(hidden_layer4, keep_prob = 1)

# output layers

out = tf.transpose(tf.add(tf.matmul(hidden_layer4, weight_out), bias_out))

In [0]:
# cost function uses mean squared error to compare deviations between model predictions and observed training 
mse = tf.reduce_mean(tf.squared_difference(out, Y))

In [0]:
#Optimizer - adapts weights and bias variables during training, calculate gradients which indicate 
# direction of change in weights and biases during training to reduce mse

# use AdamOptimizer 
optimizer = tf.train.AdamOptimizer().minimize(mse)


In [1]:
# Fitting network - training network to draw data samples, make predictions, and use placeholders X and Y 
# inputs and targets. After prediction, the model optimizes its weights and biases using mse and the Adam
# optimizer to update parameters for improved accuracy in predictions. 


net = tf.Session()
net.run(tf.global_variables_initializer())

# set up graph to visualize predictions and accuracy
plt.ion()
fig = plt.figure()
ax1 = fig.add_subplot(111)
line1, = ax1.plot(y_test)
line2, = ax1.plot(y_test*0.5)

# epochs represents number of times ran, batch size represents size of data samples fed into training network
epochs = 10
batch_size = 256
mse_train = []
mse_test = []

for e in range(epochs):
    #randomize training data
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    x_train = x_train[shuffle_indices] # data randomly selected for training 
    y_train = y_train[shuffle_indices]
    
    # training batch data
    for i in range (0, len(y_train) // batch_size):
      start = i * batch_size
      batch_x = x_train[start:start + batch_size]
      batch_y = y_train[start:start + batch_size]
    
    # run optimizer on batch data
      net.run(opt, feed_dict={X: batch_x, Y: batch_y})
      
      # graph progress over epochs
      
      if np.mod(i, 5) == 0:
        
          # Prediction 
          pred = net.run(out, feed_dict={X, X_test})
          line2.set_ydata(pred)
          plt.title('Epoch #' + str(e) + ', Batch ' + str(i))
          file_name = 'img/epoch_' + str(e) + '_batch_' + str(i) + '.jpg'
          plt.savefig(file_name)
          plt.pause(0.01)
          files.download("modelimages.png")

# Print final MSE after Training
mse_final = net.run(mse, feed_dict={X: x_test, Y: y_test})
print(mse_final)



NameError: ignored